**Dataset:** zenodo.org/records/2867216

In [69]:
import feature_calculation.build_biomarker_csv as build_biomarker_csv
import pandas as pd
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

In [32]:
build_biomarker_csv.build_csv("MDVR-KCL-ReadTextPD-Eng.csv", audio_dir="./MDVR-KCL/ReadText/PD-Eng")
build_biomarker_csv.build_csv("MDVR-KCL-ReadTextHC-Eng.csv", audio_dir="./MDVR-KCL/ReadText/HC-Eng")
build_biomarker_csv.build_csv("MDVR-KCL-ReadTextPD-Sun.csv", audio_dir="./MDVR-KCL/ReadText/PD-Sun")
build_biomarker_csv.build_csv("MDVR-KCL-ReadTextHC-Sun.csv", audio_dir="./MDVR-KCL/ReadText/HC-Sun")

Processing ID02_pd_2_0_0.wav...
Text data saved to ./MDVR-KCL/ReadText/Text Files
Preprocessed audio saved to ./MDVR-KCL/ReadText/PD-Eng/ID02_pd_2_0_0_preprocessed.wav
Alignment_Path:  ./MDVR-KCL/ReadText/Alignment Files/ID02_pd_2_0_0_preprocessed.json
Alignment data saved to ./MDVR-KCL/ReadText/Alignment Files
Processing ID04_pd_2_0_1.wav...
Text data saved to ./MDVR-KCL/ReadText/Text Files
Preprocessed audio saved to ./MDVR-KCL/ReadText/PD-Eng/ID04_pd_2_0_1_preprocessed.wav
Alignment_Path:  ./MDVR-KCL/ReadText/Alignment Files/ID04_pd_2_0_1_preprocessed.json
Alignment data saved to ./MDVR-KCL/ReadText/Alignment Files
Metrics saved to MDVR-KCL-ReadTextPD-Eng.csv
Processing ID01_hc_0_0_0.wav...
Text data saved to ./MDVR-KCL/ReadText/Text Files
Preprocessed audio saved to ./MDVR-KCL/ReadText/HC-Eng/ID01_hc_0_0_0_preprocessed.wav
Alignment_Path:  ./MDVR-KCL/ReadText/Alignment Files/ID01_hc_0_0_0_preprocessed.json
Alignment data saved to ./MDVR-KCL/ReadText/Alignment Files
Processing ID14_

In [ ]:
pd_eng = pd.read_csv("MDVR-KCL-ReadTextPD-Eng.csv", index_col="filename").sort_index()
hc_eng = pd.read_csv("MDVR-KCL-ReadTextHC-Eng.csv", index_col="filename").sort_index()
pd_sun = pd.read_csv("MDVR-KCL-ReadTextPD-Sun.csv", index_col="filename").sort_index()
hc_sun = pd.read_csv("MDVR-KCL-ReadTextHC-Sun.csv", index_col="filename").sort_index()

In [ ]:
# Acoustic features generated per (https://arxiv.org/abs/2111.10207) on MVDR
full_validation_set = pd.read_csv("/Validation Files/MDVR_KCL_reading_acoustic_features.csv")
pd_eng_validation = full_validation_set[full_validation_set["voiceID"].str.contains("PD-Eng") & full_validation_set["voiceID"].str.contains("preprocessed")]
hc_eng_validation = full_validation_set[full_validation_set["voiceID"].str.contains("HC-Eng") & full_validation_set["voiceID"].str.contains("preprocessed")]
pd_sun_validation = full_validation_set[full_validation_set["voiceID"].str.contains("PD-Sun") & full_validation_set["voiceID"].str.contains("preprocessed")]
hc_sun_validation = full_validation_set[full_validation_set["voiceID"].str.contains("HC-Sun") & full_validation_set["voiceID"].str.contains("preprocessed")]
for df in [pd_eng_validation, hc_eng_validation, pd_sun_validation, hc_sun_validation]:
    df["voiceID"] = df["voiceID"].str.split("/").str[-1].str.replace("_preprocessed", "").str.replace(".wav", "")
    df.set_index("voiceID", inplace=True)
    df.sort_index(inplace=True)


In [ ]:
pairs = [(pd_eng, pd_eng_validation), 
         (hc_eng, hc_eng_validation), 
         (pd_sun, pd_sun_validation), 
         (hc_sun, hc_sun_validation)]
for pair in pairs: